In [1]:
   print('Jupyter notebook is working!')

Jupyter notebook is working!


In [2]:
# 1. Import Libraries
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
nltk.download('stopwords')

ModuleNotFoundError: No module named 'nltk'

In [5]:
!pip install nltk


In [6]:
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
# 2. Load the Dataset
data = pd.read_csv('article50.csv')
data.head()

,source_text,plagiarized_text,label
0,Researchers have discovered a new species of b...,Scientists have found a previously unknown but...,1
1,The moon orbits the Earth in approximately 27....,Our natural satellite takes around 27.3 days t...,1
2,Water is composed of two hydrogen atoms and on...,H2O consists of 2 hydrogen atoms and 1 oxygen ...,1
3,The history of Rome dates back to 753 BC.,Rome has a long history that can be traced bac...,1
4,Pluto was once considered the ninth planet in ...,"In the past, Pluto was classified as the ninth...",1


In [8]:
# 3. Preprocess the Text Data
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

data['source_text_clean'] = data['source_text'].apply(preprocess_text)
data['plagiarized_text_clean'] = data['plagiarized_text'].apply(preprocess_text)
data[['source_text_clean', 'plagiarized_text_clean', 'label']].head()

,source_text_clean,plagiarized_text_clean,label
0,researchers discovered new species butterfly a...,scientists found previously unknown butterfly ...,1
1,moon orbits earth approximately 273 days,natural satellite takes around 273 days comple...,1
2,water composed two hydrogen atoms one oxygen atom,h2o consists 2 hydrogen atoms 1 oxygen atom,1
3,history rome dates back 753 bc,rome long history traced back 753 bc,1
4,pluto considered ninth planet solar system,past pluto classified ninth planet suns planet...,1


In [9]:
# 4. Feature Extraction (TF-IDF)
# We combine the source and plagiarized texts for each pair and extract TF-IDF features.
combined_texts = data['source_text_clean'] + ' ' + data['plagiarized_text_clean']
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(combined_texts)
y = data['label']

In [10]:
# 5. Train/Test Split and Model Training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [11]:
# 6. Model Evaluation
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))

Accuracy: 0.42857142857142855
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.43      1.00      0.60         3

    accuracy                           0.43         7
   macro avg       0.21      0.50      0.30         7
weighted avg       0.18      0.43      0.26         7

Confusion Matrix:
 [[0 4]
 [0 3]]


C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

In [12]:
# 7. Example: Predict Plagiarism for New Texts
def predict_plagiarism(source, suspect):
    s = preprocess_text(source)
    t = preprocess_text(suspect)
    combined = s + ' ' + t
    features = tfidf.transform([combined])
    pred = model.predict(features)[0]
    prob = model.predict_proba(features)[0][1]
    return pred, prob

# Example
source_example = 'The cheetah is the fastest land animal.'
suspect_example = 'No land animal is faster than the cheetah.'
pred, prob = predict_plagiarism(source_example, suspect_example)
print(f'Prediction: {"Plagiarized" if pred else "Not Plagiarized"} (Probability: {prob:.2f})')

Prediction: Plagiarized (Probability: 0.65)


In [13]:
import joblib

# Save the model and vectorizer
joblib.dump(model, 'plagiarism_model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']